In [11]:
import numpy as np
import os
import sys
import multiprocessing
import time

# PyTorch dependencies
import torch as pt
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

# local dependencies
from sembps.bps import aptbps
from modelnet40 import load_modelnet40

MAIN_PATH = os.path.join(os.sep, 'media', 'add320', 'riccardo')
MOD40_PATH = os.path.join(MAIN_PATH, 'modelnet40')

LOGS_PATH = os.path.join(MOD40_PATH, 'fft_logs')
DATA_PATH = os.path.join(MOD40_PATH, 'fft_data')

BPS_CACHE_FILE = os.path.join(DATA_PATH, 'bps_mlp_data.npz')
APTBPS_CACHE_FILE = os.path.join(DATA_PATH, 'fft_aptbps_mlp_data.npz')

N_MODELNET_CLASSES = 40

N_BPS_POINTS = 512
BPS_RADIUS = 1.7

N_CPUS = multiprocessing.cpu_count()
N_GPUS = torch.cuda.device_count()

if N_GPUS > 0:
    DEVICE = 'cuda'
    print("GPU device found...")
else:
    DEVICE = 'cpu'
    print("GPU device not found, using %d CPU(s)..." % N_CPUS)

if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)


class ShapeClassifierMLP(nn.Module):

    def __init__(self, n_features, n_classes, hsize1=512,  hsize2=512, dropout1=0.8, dropout2=0.8):
        super(ShapeClassifierMLP, self).__init__()

        self.bn0 = nn.BatchNorm1d(n_features)
        self.fc1 = nn.Linear(in_features=n_features, out_features=hsize1)
        self.bn1 = nn.BatchNorm1d(hsize1)
        self.do1 = nn.Dropout(dropout1)
        self.fc2 = nn.Linear(in_features=hsize1, out_features=hsize2)
        self.bn2 = nn.BatchNorm1d(hsize2)
        self.do2 = nn.Dropout(dropout2)
        self.fc3 = nn.Linear(in_features=hsize2, out_features=n_classes)

    def forward(self, x):

        x = self.bn0(x)
        x = self.do1(self.bn1(F.relu(self.fc1(x))))
        x = self.do2(self.bn2(F.relu(self.fc2(x))))
        x = self.fc3(x)

        return x

def fit(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()


def test(model, device, test_loader, epoch_id):
    model.eval()
    test_loss = 0
    n_test_samples = len(test_loader.dataset)
    n_correct = 0
    with pt.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            n_correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= n_test_samples
    test_acc = 100.0 * n_correct / n_test_samples
    if epoch_id % 10 == 0:
        print(
            "Epoch {} test loss: {:.4f}, test accuracy: {}/{} ({:.2f}%)".format(epoch_id, test_loss, n_correct, n_test_samples, test_acc))

    return test_loss, test_acc


def prepare_data_loaders(n_parts=2):

    if n_parts == 1:
        APTBPS_CACHE_FILE = os.path.join(DATA_PATH, 'fft_aptbps_mlp_data.npz')
    else:
        file_name = str(n_parts) + 'fft_aptbps_mlp_data.npz'
        APTBPS_CACHE_FILE = os.path.join(DATA_PATH, file_name)
    
    if not os.path.exists(APTBPS_CACHE_FILE):

        # load modelnet point clouds
        xtr, ytr, xte, yte = load_modelnet40(root_data_dir=DATA_PATH)

        # this will normalise your point clouds and return scaler parameters for inverse operation
        xtr_normalized = aptbps.normalize(xtr)
        xte_normalized = aptbps.normalize(xte)

        # this will encode your normalised point clouds with random basis of 512 points,
        # each BPS cell containing l2-distance to closest point
        start = time.time()
        print("converting data to BPS representation..")
        print("number of basis points: %d" % N_BPS_POINTS)
        print("BPS sampling radius: %f" % BPS_RADIUS)
        print("converting train..")
        xtr_bps = aptbps.fft_encode(xtr_normalized, n_bps_points=N_BPS_POINTS, n_parts=n_parts, bps_cell_type='dists', radius=BPS_RADIUS) # train
        print("converting test..")
        xte_bps = aptbps.fft_encode(xte_normalized, n_bps_points=N_BPS_POINTS, n_parts=n_parts, bps_cell_type='dists', radius=BPS_RADIUS) # test
        end = time.time()
        total_training_time = (end - start) / 60
        print("conversion finished. ")
        print("saving cache file for future runs..")

        np.savez(APTBPS_CACHE_FILE, xtr=xtr_bps, ytr=ytr, xte=xte_bps, yte=yte)

    else:
        print("loading converted data from cache..")
        data = np.load(APTBPS_CACHE_FILE)
        xtr_bps = data['xtr']
        ytr = data['ytr']
        xte_bps = data['xte']
        yte = data['yte']


    # long() converts it to long
    # these datasets contain the data laready normalized and encoded with bps
    dataset_tr = pt.utils.data.TensorDataset(pt.Tensor(xtr_bps), pt.Tensor(ytr[:, 0]).long())

    train_loader = pt.utils.data.DataLoader(dataset_tr, batch_size=512, shuffle=True)

    dataset_te = pt.utils.data.TensorDataset(pt.Tensor(xte_bps), pt.Tensor(yte[:, 0]).long())
    test_loader = pt.utils.data.DataLoader(dataset_te, batch_size=512, shuffle=True)

    return train_loader, test_loader

GPU device found...


In [12]:
import csv

def main():
    
    for i in range(2, 20):

        train_loader, test_loader = prepare_data_loaders(i)
    
        n_bps_features = train_loader.dataset[0][0].shape[0]
    
        print("defining the model..")
        model = ShapeClassifierMLP(n_features=n_bps_features, n_classes=N_MODELNET_CLASSES)
    
        optimizer = pt.optim.Adam(model.parameters(), lr=1e-3)
    
        n_epochs = 1200
        pbar = range(0, n_epochs)
        test_accs = []
        test_losses = []
    
        print("training started..")
        model = model.to(DEVICE)
        
        # Train in parallel on multiple GPUs
        model = nn.DataParallel(model)
    
        start = time.time()
    
        for epoch_idx in pbar:
            fit(model, DEVICE, train_loader, optimizer)
            if epoch_idx == 1000:
                for param_group in optimizer.param_groups:
                    print("decreasing the learning rate to 1e-4..")
                    param_group['lr'] = 1e-4
            test_loss, test_acc = test(model, DEVICE, test_loader, epoch_idx)
            test_accs.append(test_acc)
            test_losses.append(test_loss)
            if test_acc > 89.0:
                file_name = str(i) + 'fft_aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
                ckpt_path = os.path.join(LOGS_PATH, file_name)
                pt.save(model.state_dict(), ckpt_path)
                print("Model saved: %s" % ckpt_path)
            # Save every 200 epochs
            if epoch_idx % 200 == 0:
                file_name = str(i) + 'fft_aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
                ckpt_path = os.path.join(LOGS_PATH, file_name)
                pt.save(model.state_dict(), ckpt_path)
                print("Model saved: %s" % ckpt_path)
    
        accs_file_name = str(i) + 'accs_fft_aptbps_mlp_model.csv'
        accs_file_path = os.path.join(LOGS_PATH, accs_file_name)
        
        with open(accs_file_path, 'w', newline='\n') as f:
            for item in test_accs:
                f.write("%s\n" % item)
                
        losses_file_name = str(i) + 'losses_fft_aptbps_mlp_model.csv'
        losses_file_path = os.path.join(LOGS_PATH, losses_file_name)
        
        with open(losses_file_path, 'w', newline='\n') as ff:
            for item in test_losses:
                ff.write("%s\n" % item)
    
        _, test_acc = test(model, DEVICE, test_loader, n_epochs)
    
        end = time.time()
        total_training_time = (end - start) / 60
    
        print("Training finished. Test accuracy: %f . Total training time: %f minutes." % (test_acc, total_training_time))
        file_name = str(i) + 'fft_aptbps_mlp_model.h5'
        ckpt_path = os.path.join(LOGS_PATH, file_name)
    
        pt.save(model.state_dict(), ckpt_path)
    
        print("Model saved: %s" % ckpt_path)

    return


if __name__ == '__main__':
    main()


loading converted data from cache..
defining the model..
training started..
Epoch 0 test loss: -2.0993, test accuracy: 923/2468 (37.40%)
Model saved: /media/add320/riccardo/modelnet40/fft_logs/2fft_aptbps_mlp_model_epoch0.h5
Epoch 10 test loss: -6.9731, test accuracy: 1703/2468 (69.00%)
Epoch 20 test loss: -8.1926, test accuracy: 1834/2468 (74.31%)
Epoch 30 test loss: -8.7734, test accuracy: 1923/2468 (77.92%)
Epoch 40 test loss: -9.1810, test accuracy: 1937/2468 (78.48%)
Epoch 50 test loss: -9.5985, test accuracy: 1956/2468 (79.25%)
Epoch 60 test loss: -9.7256, test accuracy: 1981/2468 (80.27%)
Epoch 70 test loss: -9.9861, test accuracy: 1970/2468 (79.82%)
Epoch 80 test loss: -10.1362, test accuracy: 2006/2468 (81.28%)
Epoch 90 test loss: -10.1762, test accuracy: 1984/2468 (80.39%)
Epoch 100 test loss: -10.4030, test accuracy: 2001/2468 (81.08%)
Epoch 110 test loss: -10.6085, test accuracy: 2005/2468 (81.24%)
Epoch 120 test loss: -10.5617, test accuracy: 2017/2468 (81.73%)
Epoch 130 t

100%|██████████| 820/820 [00:09<00:00, 82.74it/s] 


converting test..
using 12 available CPUs for BPS encoding..


100%|██████████| 206/206 [00:02<00:00, 78.44it/s]


conversion finished. 
saving cache file for future runs..
defining the model..
training started..
Epoch 0 test loss: -2.0177, test accuracy: 971/2468 (39.34%)
Model saved: /media/add320/riccardo/modelnet40/fft_logs/4fft_aptbps_mlp_model_epoch0.h5
Epoch 10 test loss: -7.3482, test accuracy: 1813/2468 (73.46%)
Epoch 20 test loss: -8.5882, test accuracy: 1904/2468 (77.15%)
Epoch 30 test loss: -9.3744, test accuracy: 1939/2468 (78.57%)
Epoch 40 test loss: -9.8600, test accuracy: 1982/2468 (80.31%)
Epoch 50 test loss: -10.0860, test accuracy: 1985/2468 (80.43%)
Epoch 60 test loss: -10.3864, test accuracy: 2020/2468 (81.85%)
Epoch 70 test loss: -10.7115, test accuracy: 2031/2468 (82.29%)
Epoch 80 test loss: -10.8760, test accuracy: 2025/2468 (82.05%)
Epoch 90 test loss: -11.0226, test accuracy: 2034/2468 (82.41%)
Epoch 100 test loss: -11.1098, test accuracy: 2040/2468 (82.66%)
Epoch 110 test loss: -11.1841, test accuracy: 2048/2468 (82.98%)
Epoch 120 test loss: -11.4635, test accuracy: 2057/

...

conversion finished. 
saving cache file for future runs..
defining the model..
training started..
Epoch 0 test loss: -1.8799, test accuracy: 890/2468 (36.06%)
Model saved: /media/add320/riccardo/modelnet40/fft_logs/18fft_aptbps_mlp_model_epoch0.h5
Epoch 10 test loss: -7.0671, test accuracy: 1759/2468 (71.27%)
Epoch 20 test loss: -8.1460, test accuracy: 1836/2468 (74.39%)
Epoch 30 test loss: -8.7763, test accuracy: 1888/2468 (76.50%)
Epoch 40 test loss: -9.1154, test accuracy: 1925/2468 (78.00%)
Epoch 50 test loss: -9.4496, test accuracy: 1956/2468 (79.25%)
Epoch 60 test loss: -9.6818, test accuracy: 1970/2468 (79.82%)
Epoch 70 test loss: -9.9749, test accuracy: 1977/2468 (80.11%)
Epoch 80 test loss: -10.0979, test accuracy: 1999/2468 (81.00%)
Epoch 90 test loss: -10.4432, test accuracy: 1993/2468 (80.75%)
Epoch 100 test loss: -10.5925, test accuracy: 1987/2468 (80.51%)
Epoch 110 test loss: -10.7362, test accuracy: 2010/2468 (81.44%)
Epoch 120 test loss: -10.7136, test accuracy: 2001/24

100%|██████████| 820/820 [00:19<00:00, 42.14it/s]


converting test..
using 12 available CPUs for BPS encoding..


100%|██████████| 206/206 [00:04<00:00, 41.47it/s]

100%|██████████| 205/205 [00:04<00:00, 41.37it/s]

100%|██████████| 206/206 [00:05<00:00, 40.57it/s]


conversion finished. 
saving cache file for future runs..
defining the model..
training started..
Epoch 0 test loss: -1.9350, test accuracy: 864/2468 (35.01%)
Model saved: /media/add320/riccardo/modelnet40/fft_logs/19fft_aptbps_mlp_model_epoch0.h5
Epoch 10 test loss: -7.1492, test accuracy: 1766/2468 (71.56%)
Epoch 20 test loss: -8.2548, test accuracy: 1871/2468 (75.81%)
Epoch 30 test loss: -8.9192, test accuracy: 1904/2468 (77.15%)
Epoch 40 test loss: -9.3137, test accuracy: 1937/2468 (78.48%)
Epoch 50 test loss: -9.5779, test accuracy: 1961/2468 (79.46%)
Epoch 60 test loss: -9.9538, test accuracy: 1981/2468 (80.27%)
Epoch 70 test loss: -10.2035, test accuracy: 1984/2468 (80.39%)
Epoch 80 test loss: -10.4206, test accuracy: 1979/2468 (80.19%)
Epoch 90 test loss: -10.4465, test accuracy: 1989/2468 (80.59%)
Epoch 100 test loss: -10.9416, test accuracy: 1996/2468 (80.88%)
Epoch 110 test loss: -10.9603, test accuracy: 2006/2468 (81.28%)
Epoch 120 test loss: -10.9936, test accuracy: 2000/2

In [ ]:
import csv

def main():
    
    for i in range(3, 4):

        train_loader, test_loader = prepare_data_loaders(i)
    
        n_bps_features = train_loader.dataset[0][0].shape[0]
    
        print("defining the model..")
        model = ShapeClassifierMLP(n_features=n_bps_features, n_classes=N_MODELNET_CLASSES)
    
        optimizer = pt.optim.Adam(model.parameters(), lr=1e-3)
    
        n_epochs = 1200
        pbar = range(0, n_epochs)
        test_accs = []
        test_losses = []
    
        print("training started..")
        model = model.to(DEVICE)
        
        # Train in parallel on multiple GPUs
        model = nn.DataParallel(model)
    
        start = time.time()
    
        for epoch_idx in pbar:
            fit(model, DEVICE, train_loader, optimizer)
            if epoch_idx == 1000:
                for param_group in optimizer.param_groups:
                    print("decreasing the learning rate to 1e-4..")
                    param_group['lr'] = 1e-4
            test_loss, test_acc = test(model, DEVICE, test_loader, epoch_idx)
            test_accs.append(test_acc)
            test_losses.append(test_loss)
            if test_acc > 89.0:
                file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
                ckpt_path = os.path.join(LOGS_PATH, file_name)
                pt.save(model.state_dict(), ckpt_path)
                print("Model saved: %s" % ckpt_path)
            #if epoch_idx % 100 == 0:
            #    file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
            #    ckpt_path = os.path.join(LOGS_PATH, file_name)
            #    pt.save(model.state_dict(), ckpt_path)
            #    print("Model saved: %s" % ckpt_path)
        
        accs_file_name = str(i) + 'accs_aptbps_mlp_model.csv'
        accs_file_path = os.path.join(LOGS_PATH, accs_file_name)
        
        with open(accs_file_path, 'w', newline='\n') as f:
            for item in test_accs:
                f.write("%s\n" % item)
                
        losses_file_name = str(i) + 'losses_aptbps_mlp_model.csv'
        losses_file_path = os.path.join(LOGS_PATH, losses_file_name)
        
        with open(losses_file_path, 'w', newline='\n') as ff:
            for item in test_losses:
                ff.write("%s\n" % item)
            
        _, test_acc = test(model, DEVICE, test_loader, n_epochs)
    
        end = time.time()
        total_training_time = (end - start) / 60
    
        print("Training finished. Test accuracy: %f . Total training time: %f minutes." % (test_acc, total_training_time))
        file_name = str(i) + 'aptbps_mlp_model.h5'
        ckpt_path = os.path.join(LOGS_PATH, file_name)
    
        pt.save(model.state_dict(), ckpt_path)
    
        print("Model saved: %s" % ckpt_path)

    return


if __name__ == '__main__':
    main()
